# Toronto city Neighborhood Analysis - Assignment
## Part 1 Creating Toronto Neighborhood DataFrame and filling it with webscrapped data

Packages will be imported as needed, not all at once at the beginning

In [46]:
import pandas as pd
import numpy as np

!pip install bs4
from bs4 import BeautifulSoup 

!pip install requests
import requests



In [48]:
neighborhood_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
neighborhood_data = requests.get(neighborhood_url).text
soup = BeautifulSoup(neighborhood_data, 'html5lib')
tables_list = soup.find_all('table')

print("There are {} tables in the webpage".format(len(tables_list)))

There are 3 tables in the webpage


In [49]:
for ind,table in enumerate(tables_list):
    if ("M1A" in str(table)):
        table_ind = ind
print("The table we want is of index {}".format(table_ind))


The table we want is of index 0


In [52]:
neigh_table = tables_list[0]
print(neigh_table.prettify())

<table cellpadding="2" cellspacing="0" rules="all" style="width:100%; border-collapse:collapse; border:1px solid #ccc;">
 <tbody>
  <tr>
   <td style="width:11%; vertical-align:top; color:#ccc;">
    <p>
     <b>
      M1A
     </b>
     <br/>
     <span style="font-size:85%;">
      <i>
       Not assigned
      </i>
     </span>
    </p>
   </td>
   <td style="width:11%; vertical-align:top; color:#ccc;">
    <p>
     <b>
      M2A
     </b>
     <br/>
     <span style="font-size:85%;">
      <i>
       Not assigned
      </i>
     </span>
    </p>
   </td>
   <td style="width:11%; vertical-align:top;">
    <p>
     <b>
      M3A
     </b>
     <br/>
     <span style="font-size:85%;">
      <a href="/wiki/North_York" title="North York">
       North York
      </a>
      <br/>
      (
      <a href="/wiki/Parkwoods" title="Parkwoods">
       Parkwoods
      </a>
      )
     </span>
    </p>
   </td>
   <td style="width:11%; vertical-align:top;">
    <p>
     <b>
      M4A
     </b>
 

After analysing the html context of the Toronto neighborhoods table, I came to the conclusion that each data is in a cell not in a row
The structure of each data cell is following:

In [53]:
# Creating an empty list to store the data
table_contents = []

#Iterating through the each table cell that is inside the <td> tag
for row in neigh_table.find_all("td"):
    # Creating an empty cell dict
    cell = {}
    
    # Checking the condition of cell containing "Not assigned"
    if row.span.text == 'Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[0:3] # Getting the PostalCode value, which is the first 3 char values in each cell
        cell['Borough'] = row.span.text.split('(')[0] # Getting Borough data 
        #Getting the Neighborhood data
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ') 
        
        #Adding the cell values into the list
        table_contents.append(cell)
        
# Transforming the list into the dataframe 
neigh_df=pd.DataFrame(table_contents)

# Make adjustments
neigh_df['Borough']=neigh_df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

print("Shape of table is {}".format(neigh_df.shape))
neigh_df.head(5)

Shape of table is (103, 3)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


## Part 2: Getting the geo data for each Borough

In [58]:
!pip install pgeocode
import pgeocode

In [68]:
postalCodes = neigh_df['PostalCode'].tolist() # Converting postal codes to the list

geolocator = pgeocode.Nominatim('ca')# Define the geolocator

latitudes = [] # Creating the list for latitude data
longitudes = [] # Creating the list for longitude data


for i, postalCode in enumerate(postalCodes): # Iterating through the postal codes to get their latitude and longitude
    
    # Getting the specific location
    g = geolocator.query_postal_code(postalCode)
    
    # Get latitude and longitude
    if not g.empty:
        latitudes.append(g.latitude)
        longitudes.append(g.longitude)
    else:
        latitudes.append("Not found")
        longitudes.append("Not found")

In [69]:
neigh_df['Latitude'] = latitudes
neigh_df['Longitude'] = longitudes
neigh_df.head(5)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
4,M7A,Queen's Park,Ontario Provincial Government,43.6641,-79.3889


In [70]:
neigh_df.drop(76, inplace=True)
neigh_df.reset_index(inplace=True)

In [71]:
print("The shape of the dataframe is {}".format(neigh_df.shape))

The shape of the dataframe is (102, 6)


## Part 3: Explore the data
### Part 3.1: Visualize the points on a map
#### Import folium package for displaying map and other packages for color visualization

In [72]:
!pip install folium 
import folium # map rendering library

In [89]:
# For coloring the clusters
import matplotlib.cm as cm
import matplotlib.colors as colors

In [74]:
!pip install sklearn
from sklearn.cluster import KMeans # KMeans clustering package

  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1320 sha256=911cc0bca5ab8a4136e7e71f0cf1c56a0adb2cb9a6f282d7ac8beb2e717d2782
  Stored in directory: c:\users\beket\appdata\local\pip\cache\wheels\22\0b\40\fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897
Successfully built sklearn


In [75]:
{
    "tags": [
        "hide-input",
    ]
}
CLIENT_ID = 'G1ZNGFWYJGBC3NBIMUUXMNKXK03CKMQJO44AWAAWQQAJZ41J' # your Foursquare ID
CLIENT_SECRET = '550RG0M2CN4D5DDSJLEAWRQBCVU0FQBYFK4HLLXN4IAZD44O' # your Foursquare Secret
VERSION = '20210404' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [76]:
# Defining getNearbyVenues function to get nearby venues based on latitude and longitude values
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    print('Done')
    return(nearby_venues)

In [77]:
# Executing the getNearbyVenues function on our toronto neighborhood dataframe
toronto_venues = getNearbyVenues(names=neigh_df['PostalCode'] + ' - ' + neigh_df['Neighborhood'],
                                 latitudes = neigh_df['Latitude'],
                                 longitudes = neigh_df['Longitude'])

M3A - Parkwoods
M4A - Victoria Village
M5A - Regent Park, Harbourfront
M6A - Lawrence Manor, Lawrence Heights
M7A - Ontario Provincial Government
M9A - Islington Avenue
M1B - Malvern, Rouge
M3B - Don Mills North
M4B - Parkview Hill, Woodbine Gardens
M5B - Garden District, Ryerson
M6B - Glencairn
M9B - West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
M1C - Rouge Hill, Port Union, Highland Creek
M3C - Don Mills South
M4C - Woodbine Heights
M5C - St. James Town
M6C - Humewood-Cedarvale
M9C - Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
M1E - Guildwood, Morningside, West Hill
M4E - The Beaches
M5E - Berczy Park
M6E - Caledonia-Fairbanks
M1G - Woburn
M4G - Leaside
M5G - Central Bay Street
M6G - Christie
M1H - Cedarbrae
M2H - Hillcrest Village
M3H - Bathurst Manor, Wilson Heights, Downsview North
M4H - Thorncliffe Park
M5H - Richmond, Adelaide, King
M6H - Dufferin, Dovercourt Village
M1J - Scarborough Village
M2J - Fairview, Henry Farm, Oriole
M3J - Nor

In [78]:
print('Shape of toronto_venues is {}'.format(toronto_venues.shape))
toronto_venues.head(15)

Shape of toronto_venues is (2156, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A - Parkwoods,43.7545,-79.3300,Brookbanks Park,43.751976,-79.332140,Park
1,M3A - Parkwoods,43.7545,-79.3300,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,M3A - Parkwoods,43.7545,-79.3300,Yorkmills Wellness & Spa,43.756800,-79.325346,Spa
3,M4A - Victoria Village,43.7276,-79.3148,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,M4A - Victoria Village,43.7276,-79.3148,Portugril,43.725819,-79.312785,Portuguese Restaurant
5,M4A - Victoria Village,43.7276,-79.3148,Tim Hortons,43.725517,-79.313103,Coffee Shop
6,M4A - Victoria Village,43.7276,-79.3148,Eglinton Ave E & Sloane Ave/Bermondsey Rd,43.726086,-79.313620,Intersection
7,M4A - Victoria Village,43.7276,-79.3148,Pizza Nova,43.725824,-79.312860,Pizza Place
8,M4A - Victoria Village,43.7276,-79.3148,Wigmore Park,43.731023,-79.310771,Park
9,"M5A - Regent Park, Harbourfront",43.6555,-79.3626,Tandem Coffee,43.653559,-79.361809,Coffee Shop


In [79]:
# How my venues in each neighborhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"M1B - Malvern, Rouge",1,1,1,1,1,1
"M1E - Guildwood, Morningside, West Hill",34,34,34,34,34,34
M1G - Woburn,1,1,1,1,1,1
M1H - Cedarbrae,4,4,4,4,4,4
M1J - Scarborough Village,3,3,3,3,3,3
...,...,...,...,...,...,...
M9N - Weston,2,2,2,2,2,2
M9P - Westmount,11,11,11,11,11,11
"M9R - Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens",11,11,11,11,11,11


In [80]:
# number of unique categories
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 260 uniques categories.


In [81]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [83]:
# Grouping by neighborhoods 
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head(10)

,Neighborhood,Yoga Studio,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"M1B - Malvern, Rouge",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"M1E - Guildwood, Morningside, West Hill",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M1G - Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M1H - Cedarbrae,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M1J - Scarborough Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,"M1K - Kennedy Park, Ionview, East Birchmount Park",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,"M1L - Golden Mile, Clairlea, Oakridge",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,"M1M - Cliffside, Cliffcrest, Scarborough Villa...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,"M1N - Birch Cliff, Cliffside West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,"M1P - Dorset Park, Wexford Heights, Scarboroug...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Part 3.2: Clustering Neighborhoods

In [84]:
n_clusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([4, 3, 2, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 0, 3, 0, 3, 1,
       0, 1, 1, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 0, 1, 0,
       0, 3, 0, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 4, 4, 3, 3, 3,
       3, 3, 3, 3, 3, 0, 0, 0, 3, 3, 3, 3, 3, 0, 1, 3, 3, 3, 3, 3, 3, 1,
       3, 0, 3, 3, 3, 3, 1, 3, 3, 3, 3])

In [85]:
# Add labels to the clusters
toronto_grouped_clustering2 = toronto_grouped[:]
toronto_grouped_clustering2['ClusterNumber'] = kmeans.labels_
toronto_grouped_clustering2.head(5)

<ipython-input-85-7118f68f0e90>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  toronto_grouped_clustering2['ClusterNumber'] = kmeans.labels_


,Neighborhood,Yoga Studio,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,ClusterNumber
0,"M1B - Malvern, Rouge",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
1,"M1E - Guildwood, Morningside, West Hill",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
2,M1G - Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
3,M1H - Cedarbrae,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
4,M1J - Scarborough Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [86]:
# Merging

toronto_merged = neigh_df[:]
toronto_merged['Neighborhood'] = toronto_merged['PostalCode'] + " - " + toronto_merged['Neighborhood']

toronto_merged = toronto_merged.join(toronto_grouped_clustering2.set_index('Neighborhood'), on='Neighborhood', how='right', lsuffix='_left', rsuffix='_right')

toronto_merged.head(10) # check the last columns!

,index,PostalCode,Borough,Neighborhood,Latitude,Longitude,Yoga Studio,ATM,Accessories Store,Afghan Restaurant,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,ClusterNumber
0,0,M3A,North York,M3A - Parkwoods,43.7545,-79.3300,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,0.00,0.0,0.0,0.00,0.0,0.000000,0.000000,1
1,1,M4A,North York,M4A - Victoria Village,43.7276,-79.3148,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,0.00,0.0,0.0,0.00,0.0,0.000000,0.000000,3
2,2,M5A,Downtown Toronto,"M5A - Regent Park, Harbourfront",43.6555,-79.3626,0.043478,0.0,0.000000,0.0,...,0.000000,0.0,0.00,0.0,0.0,0.00,0.0,0.000000,0.000000,3
3,3,M6A,North York,"M6A - Lawrence Manor, Lawrence Heights",43.7223,-79.4504,0.000000,0.0,0.014286,0.0,...,0.014286,0.0,0.00,0.0,0.0,0.00,0.0,0.000000,0.042857,3
4,4,M7A,Queen's Park,M7A - Ontario Provincial Government,43.6641,-79.3889,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,0.00,0.0,0.0,0.00,0.0,0.034483,0.000000,3
5,5,M9A,Etobicoke,M9A - Islington Avenue,43.6662,-79.5282,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,0.00,0.0,0.0,0.00,0.0,0.000000,0.000000,0
6,6,M1B,Scarborough,"M1B - Malvern, Rouge",43.8113,-79.1930,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,0.00,0.0,0.0,0.00,0.0,0.000000,0.000000,4
7,7,M3B,North York,M3B - Don Mills North,43.7450,-79.3590,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,0.00,0.0,0.0,0.00,0.0,0.000000,0.000000,1
8,8,M4B,East York,"M4B - Parkview Hill, Woodbine Gardens",43.7063,-79.3094,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,0.00,0.0,0.0,0.00,0.0,0.000000,0.000000,3
9,9,M5B,Downtown Toronto,"M5B - Garden District, Ryerson",43.6572,-79.3783,0.000000,0.0,0.000000,0.0,...,0.010000,0.0,0.01,0.0,0.0,0.01,0.0,0.000000,0.000000,3


In [90]:
# Let's define some colors

x = np.arange(n_clusters)
colors_array = cm.rainbow(np.linspace(0, 1, len(x)))
colors_array
rainbow = [colors.rgb2hex(i) for i in colors_array]
rainbow

['#8000ff', '#00b5eb', '#80ffb4', '#ffb360', '#ff0000']

In [92]:
# Let's plot it!

lat_toronto = 43.6532
long_toronto = -79.3832

# create map
toronto_map_clusters = folium.Map(location=[lat_toronto, long_toronto], zoom_start=11)

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['ClusterNumber']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(toronto_map_clusters)
       
toronto_map_clusters